<a href="https://colab.research.google.com/github/mmahalea19/NLP/blob/master/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
NOTE!!! Keep the same folder structure W

SyntaxError: invalid syntax (<ipython-input-10-1f500f859c75>, line 1)

In [ ]:
Task 1

In [35]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import pandas as pd
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]
    all_words = []
    for mail in emails:
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
 
    dictionary1 = Counter(all_words)
    # Paste code for non-word removal here(code snippet is given below)
    list_to_remove = list(dictionary1)
    
    for item in list_to_remove:
        if item.isalpha() == False:
            del dictionary1[item]
        elif len(item) == 1:
            del dictionary1[item]
    dictionary1 = dictionary1.most_common(3000)
    return dictionary1
def reduceFeatures(raw,nrFeats):
    #min max strategy
    x=True
    while(raw.shape[1]>nrFeats):
        sums=[sum(x) for x in zip(*raw)]
        index=-1
        if x:
            index=np.argmin(sums)
        else:
            index=np.argmax(sums)
        x=not x
        raw=np.delete(raw,index,1)
    return raw
        
        
def extract_my_features(mail_dir):
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    docID = 0;
    lines=[]
    
    
        
    # settings that you use for count vectorizer will go here
   
    for fil in files:
        with open(fil) as fi:
            for i,line in enumerate(fi):
                if i == 2:
                    lines.append(line)
    # lines=["the house had a tiny little mouse",
    #   "the cat saw the mouse",
    #   "the mouse ran away from the house",
    #   "the cat finally ate the mouse",
    #   "the end of the mouse story"]
     
    tfidf_vectorizer=TfidfVectorizer(use_idf=True)
     
    # just send in all your docs here
    tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(lines)
    first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
 
# place tf-idf values in a pandas data frame
    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
    df.sort_values(by=["tfidf"],ascending=False)
    
    return tfidf_vectorizer_vectors
def LDA_PCA(raw,option,nr_components,labels=None):
    if option is "pca":
        pca = PCA(n_components=nr_components)
        x_pca = pca.fit_transform(raw)
        return x_pca
    if option is "lda":
        lda = LDA(n_components=nr_components)
        x_lda=lda.fit_transform(raw,labels)
        return x_lda
        
def extract_features(mail_dir):
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    docID = 0;
    for fil in files:
      with open(fil) as fi:
        for i,line in enumerate(fi):
          if i == 2:
            words = line.split()
            for word in words:
              wordID = 0
              for i,d in enumerate(dictionary):
                if d[0] == word:
                  wordID = i
                  features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1
    return features_matrix

train_dir = 'Datasets/ling-spam/train-mails'
features=extract_my_features(train_dir)
features_array=features.toarray()
pca_feat=LDA_PCA(features_array,"pca",2)
train_labels = np.ones(20)
train_labels[0:9] = 0
lda_feat=LDA_PCA(features_array,"lda",2,train_labels)
reduced_features=reduceFeatures(features_array,10)
print(reduced_features)

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1682, 2 - 1) = 1 components.
  if self.classes_.size == 2:  # treat binary case as a special case
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  def transform(self, X):


KeyboardInterrupt: 

In [11]:


import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

# Create a dictionary of words with its frequency
 
train_dir = 'Datasets/ling-spam/train-mails'
dictionary=make_Dictionary(train_dir) 
# Prepare feature vectors per training mail and its labels
 
train_labels = np.ones(20)
train_labels[0:9] = 0
train_matrix = extract_features(train_dir)
 
# Training SVM and Naive bayes classifier

 
model1 = MultinomialNB()
model2 = LinearSVC()
model3 = tree.DecisionTreeClassifier()
model4 = RandomForestClassifier(max_depth=2, random_state=0)

model1.fit(train_matrix,train_labels)
model2.fit(train_matrix,train_labels)
model3.fit(train_matrix, train_labels)
model4.fit(train_matrix, train_labels)

 
# Test the unseen mails for Spam
test_dir = './Datasets/ling-spam/test-mails/'
test_matrix = extract_features(test_dir)
test_labels = np.ones(20)
test_labels[0:9] = 0
result1 = model1.predict(test_matrix)
result2 = model2.predict(test_matrix)
print (confusion_matrix(test_labels,result1))
print (confusion_matrix(test_labels,result2))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[[2 7]
 [7 4]]
[[ 3  6]
 [10  1]]


Task 3


In [12]:
from itertools import permutations 
from random import randint
class mock_class:
    def predict(self,s):
        return randint(0,1)
        
def majority_voting(classifiers,classifier_labels,sample):
    results=[]
    classifier_order=range(0,len(classifiers)-1);
    k=3
    combinations=permutations(classifier_order,k)
    for index,i in enumerate(list(combinations)):
        majority_vote=classifiers[i[0]].predict(sample)+classifiers[i[1]].predict(sample)+classifiers[i[2]].predict(sample);
        final_result=round(majority_vote/k);
        results.append(final_result)
        print("The final result for the classifiers {}|{}|{}={}".format(classifier_labels[i[0]],classifier_labels[i[1]],classifier_labels[i[2]],final_result))
fake1=mock_class()
fake2=mock_class()
fake3=mock_class()
fake4=mock_class()
classSet=[fake1,fake2,fake3,fake4]
classLabels=range(1,4)
x=3
majority_voting(classSet,classLabels,x)

The final result for the classifiers 1|2|3=0
The final result for the classifiers 1|3|2=1
The final result for the classifiers 2|1|3=0
The final result for the classifiers 2|3|1=1
The final result for the classifiers 3|1|2=1
The final result for the classifiers 3|2|1=0


NOTE!!! Keep the same folder structure as described below, when downloading the Enron dataset:

Enron_PRE

Enron_RAW

Task 4-remove stopwords and others


In [13]:
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize ,RegexpTokenizer
import string as string
def removeWords(phrases,options):
  print("Initial")
  print(phrases)
  if("link" in options):
    phrases=[re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", phrase) for phrase in phrases]
    print("After link")
    print(phrases)
  if "symbol" in options:
    whitelist = string.ascii_letters + string.digits + ' '
    symbolRemoved=[]
    for phrase in phrases:
      phrase1=''.join(list(map(lambda cha: cha if cha  in whitelist else ' ',phrase)))
      symbolRemoved.append(phrase1)
    print("After symbol removal")

    phrases=symbolRemoved
    print(phrases)


  tokenized=[word_tokenize(phrase) for phrase in phrases] #split in individual words
  print(tokenized)
  if "stopword" in options:
    stop_words = set(stopwords.words('english')) #get set of stopwords
    filtered=list(map(lambda phrase:[w for w in phrase if w not in stop_words],tokenized)) # remove words that appear in the stopwords set
    print("After stopword removal")

    tokenized=filtered
    #phrases=''.join(filtered)
    phrases=[' '.join(x) for x in filtered]
    print(phrases)
    print("Done")

removeWords(["This is-a really nice day to. ? swim in the ocean","Hello  @my friends https://mdshasdasdhas carte"],["stopword","symbol","link"])

Initial
['This is-a really nice day to. ? swim in the ocean', 'Hello  @my friends https://mdshasdasdhas carte']
After link
['This is-a really nice day to. ? swim in the ocean', 'Hello  @my friends   carte']
After symbol removal
['This is a really nice day to    swim in the ocean', 'Hello   my friends   carte']
[['This', 'is', 'a', 'really', 'nice', 'day', 'to', 'swim', 'in', 'the', 'ocean'], ['Hello', 'my', 'friends', 'carte']]
After stopword removal
['This really nice day swim ocean', 'Hello friends carte']
Done


[nltk_data] Downloading package stopwords to /home/mihai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mihai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
